In [1]:
from langchain.document_loaders import PyPDFLoader
#https://stphilipwestbrook.co.uk/wp-content/uploads/2020/03/Alice-in-Wonderland-Story.pdf
loader = PyPDFLoader("Alice-in-Wonderland-Story.pdf")
pdfpages = loader.load_and_split()

In [2]:
import os
os.environ['OPENAI_API_KEY'] = ''

In [3]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

pdfstore = FAISS.from_documents(pdfpages, embedding=OpenAIEmbeddings())

C:\Users\91798\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [4]:
retriever = pdfstore.as_retriever()

In [5]:
retrieved_docs = retriever.invoke("How did Alice reach Wonderland?")
print(retrieved_docs[0].page_content)

Suddenly, Alice felt herself falling.
For a long time, she tumbled down the rabbit hole.
Finally, she landed with a bump in a long hallway 
which had a tiny door at the end of it.


In [6]:
retriever1 = pdfstore.as_retriever()

In [7]:
retrieved_docs1 = retriever1.invoke("What strange things did Alice see?")
print(retrieved_docs1[0].page_content)

“I had a strange dream,” said Alice.
Alice told her sister all about her adventure. 
Before long, Alice realised that she hadn’t eaten for hours 
and she ran inside for something to eat.


In [8]:
retrieved_docs = retriever.invoke("What was different about the rabbit that Alice saw?")
print(retrieved_docs[0].page_content)

Alice followed the rabbit to the hole.
She peered through the entrance but she could no 
longer see the rabbit.
She squeezed her head and shoulders into the hole. 
Then, she wriggled her body a little further…


In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

question = "What was different about the rabbit that Alice saw?"
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=pdfstore.as_retriever(), llm=llm
)

unique_docs = retriever_from_llm.get_relevant_documents(query=question)
print("Number of unique documents:", len(unique_docs))

C:\Users\91798\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Number of unique documents: 4


In [10]:
unique_docs

[Document(page_content='Alice followed the rabbit to the hole.\nShe peered through the entrance but she could no \nlonger see the rabbit.\nShe squeezed her head and shoulders into the hole. \nThen, she wriggled her body a little further…', metadata={'source': 'Alice-in-Wonderland-Story.pdf', 'page': 3}),
 Document(page_content="Suddenly, a small, white rabbit raced past.\nThe rabbit was wearing clothes! \nHe pulled a watch from his pocket to check the time.\nHe cried, “Oh dear! Oh no! I'm late! I’m late!”\nThe rabbit shook his head and then disappeared down \na rabbit hole.", metadata={'source': 'Alice-in-Wonderland-Story.pdf', 'page': 2}),
 Document(page_content='Suddenly, Alice felt herself falling.\nFor a long time, she tumbled down the rabbit hole.\nFinally, she landed with a bump in a long hallway \nwhich had a tiny door at the end of it.', metadata={'source': 'Alice-in-Wonderland-Story.pdf', 'page': 4}),
 Document(page_content="“I'm far too big to fit through such a little door,”